In [1]:
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
def spider_page(soup):
    df_company = {}
    
    df_company['Name'] = soup.find('h1', class_='header-company-title').get_text(strip=True)
    df_company['Ticker'] = soup.find('div', class_='header-company-ticker').get_text(strip=True)[1:-1]
    
    # Rating Score
    rating_div = soup.find('div', class_='ratingdata-company-rating')
    class_attr = rating_div['class']
    rating = class_attr[-1].split('-')[-1]
    df_company["Score"] = rating
    
    # History Rating Score
    his_score_g = soup.find('g', class_='highcharts-data-labels highcharts-series-0 highcharts-line-series')
    his_score_list = [text_tag.text.strip() for text_tag in his_score_g.find_all('text')]
    
    his_date_g = soup.findAll('g', {'class': 'highcharts-axis-labels highcharts-xaxis-labels', 'data-z-index': '7'})[-1]
    his_date_list = [text_tag.text.strip() for text_tag in his_date_g.find_all('text')]
    
    his_list = list(zip(his_date_list, his_score_list))
    df_company["History_Score"] = his_list
    
    # Compare to peers
    for i in range(len(Compare_to_peers)):
        spans = soup.findAll('div', {'class': 'comparison-column col-xl-4'})[i].find_all('span')
        text_list = [span.get_text(strip=True) for span in spans]
        df_company[Compare_to_peers[i]] = text_list
    
    # Four involvements
    wrapper_divs = soup.find_all('div', class_='col-sm-6 col-12 business-involvement-wrapper')
    temp = []
    for wrapper in wrapper_divs:
        status_div = wrapper.find('div', class_='business-involvement-not-involved') or \
                     wrapper.find('div', class_='business-involvement-involved')  # 假设有两种 class 状态
        if status_div:
            status_text = status_div.get_text(strip=True)
            temp.append(status_text)
    if len(temp) == 0:
        df_company['Banned_Controversial_Weapons'] = 'NOT RESEARCHED'
        df_company['Gambling'] = 'NOT RESEARCHED'
        df_company['Tobacco'] = 'NOT RESEARCHED'
        df_company['Alcohol'] = 'NOT RESEARCHED'
    else:
        df_company['Banned_Controversial_Weapons'] = temp[0]
        df_company['Gambling'] = temp[1]
        df_company['Tobacco'] = temp[2]
        df_company['Alcohol'] = temp[3]
    
    # Decarbonization_Target
    Decarbonization_Target_rows = soup.find_all('div', class_='decarbonization-target-row')
    temp = []
    for row in Decarbonization_Target_rows:
        p_tags = row.find_all('p')
        if p_tags:
            last_p_text = p_tags[-1].get_text(strip=True)
            temp.append(last_p_text)
    if len(temp)!=5:
        df_company['Decarbonization_Target'] = temp[0]
        df_company['Decarbonization_Target_ITR'] = temp[1]
    else:
        temp[4] = temp[4].replace('\n\t\t\t\t\t\tp.a.','p.a.')
        df_company['Decarbonization_Target'] = temp[0]
        df_company['Decarbonization_Target_ITR'] = temp[1]
        df_company['Decarbonization_Target_Year'] = temp[2]
        df_company['Decarbonization_Target_Comprehensiveness'] = temp[3]
        df_company['Decarbonization_Target_Ambition'] = temp[4]
    
    # Controversies
    Controversies_divs = soup.find_all('div', class_=lambda x: x and 'controversy' in x)
    for div in Controversies_divs:
        text = ("Controversies_" + div.get_text(strip=True)).replace(' ','_')
        class_attr = div['class']
        for class_name in class_attr:
            if 'with-performance-flag' in class_name:
                suffix = class_name.split('-')[-1]
                df_company[text] = suffix
                break  # 假设每个 div 只有一个相关 class
    print(df_company['Name'],':',df_company['Score'],'Finished')            
    return df_company

In [3]:
# def wait_for_personal_info_form():
#     try:
#         # 增加等待时间
#         personal_info_form = WebDriverWait(driver, 10).until(
#             EC.presence_of_element_located((By.ID, 'yui3-widget-hd modal-header'))
#         )
#         if personal_info_form:
#             print("Personal info form detected. Please fill out the form and press Enter to continue...")
#             input("Press Enter after completing the form...")
#             form_status = 1
#     except Exception as e:
#         # 输出当前页面的HTML
#         with open("page_source.html", "w") as f:
#             f.write(driver.page_source)
#         print("No personal info form detected. Saved page source to 'page_source.html'.")
#         print(f"Error: {e}")

In [4]:
def search_and_get_soup(Company_name):
    global count
    print(Company_name)
    # 找到搜索框元素，并输入公司名称
    search_box = driver.find_element(By.ID, '_esgratingsprofile_keywords')
    search_box.clear()  # 清空搜索框
    search_box.send_keys(Company_name)
    print("Typing search box")
    search_box.send_keys(Keys.RETURN)
    print("Return search box")


    # 等待搜索结果加载
    time.sleep(5)
    
    try:
        # 尝试找到搜索提示的第一个结果并点击
        first_result = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'ul.ui-autocomplete li.ui-menu-item'))
        )
        if  count == 3: #Company_name == rest_companies[3] or Company_name ==rest_companies[4]:
            print('Sleep for 100s')
            time.sleep(130)
            print('Sleep end')
            count += 1
            # 爬取选择后的页面内容
            content = driver.page_source
            soup = BeautifulSoup(content, 'html.parser')
            return soup
        else:
            first_result.click()
            print("CLICK!!!!!!!!!!!!!!!!!!")
            count += 1
            print(count)
            # if form_status != 1:
            #     wait_for_personal_info_form()
        
            # 等待页面加载
            time.sleep(5)
        
            # 爬取选择后的页面内容
            content = driver.page_source
            soup = BeautifulSoup(content, 'html.parser')
            return soup
    
    except Exception as e:
        # 如果没有搜索提示结果
        skipped_companies.append(Company_name)
        print(f"No search suggestions for keyword '{Company_name}', skipping.")
        # print(f"Error: {e}")
        return None


In [5]:
def main(rest_companies):
    try:
        close_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'button#onetrust-accept-btn-handler'))
        )
        close_button.click()
        time.sleep(2)  # 等待弹窗关闭
    except Exception as e:
        print("No cookie banner to close.")
    
    # 显式等待，直到指定的输入框元素加载完成
    WebDriverWait(driver, 5).until(
        EC.presence_of_element_located((By.ID, '_esgratingsprofile_keywords'))
    )
    # print("Starting")
    random.shuffle(rest_companies)
    # 遍历所有公司
    for company_name in rest_companies:
        company_soup = search_and_get_soup(company_name)
        if company_soup:
            company_data = spider_page(company_soup)
            Found_companies.append(company_data)       
    return company_soup

In [6]:
def Check_Results(result_sofar):
    if len(Found_companies) == 0:
        rc = rest_companies
        return rc,result_sofar
    
    else:
        df = pd.DataFrame(Found_companies)
        # 合并df1和df2，保留匹配的行
        # merged_df = df.merge(Companies[['Ticker']], left_on='Ticker', right_on='Ticker', how='inner')
        merged_df = df.merge(Companies[['Ticker']], left_on='Ticker', right_on='Ticker', how='inner')
        
        # 删除全为空的列
        cleaned_df = merged_df.dropna(axis=1, how='all')
        # 将包含列表的列转换为字符串，以便可以删除重复行
        for col in cleaned_df.columns:
            if cleaned_df[col].apply(type).eq(list).any():
                cleaned_df[col] = cleaned_df[col].apply(str)
        
        # 删除重复行
        result_df = cleaned_df.drop_duplicates()
        result_df = pd.concat([result_df, result_sofar], axis=0, join='outer', ignore_index=True).drop_duplicates()
        # 合并df1和df2，标记出df1中的TickerKey在df2中是否存在
        merged_df = Companies[['Ticker','Name']].merge(result_df, left_on='Ticker', right_on='Ticker', how='left', indicator=True)
        
        # 提取在df2中找不到的行
        rc = merged_df[merged_df['_merge'] == 'left_only']['Name_x'].tolist()
        rc = merged_df[merged_df['_merge'] == 'left_only']['Ticker'].tolist()
        
        # result_df.to_csv('MSCI-ESG-Scores-Russel-3000-sofar.csv', index=False)
        result_df.to_csv('MSCI-ESG-Scores-FTSE-Allshares.csv', index=False)
        print('Saved Result sofar')
        return rc, result_df

In [7]:
# 启动Chrome浏览器并使用指定的用户数据目录和配置文件
options = webdriver.ChromeOptions()

# 增加调试输出
print("Setting up ChromeDriver service...")
service = Service(ChromeDriverManager().install())
print("Service set up complete.")

# 启动WebDriver服务
print("Starting ChromeDriver with options...")

url = 'https://www.msci.com/our-solutions/esg-investing/esg-ratings-climate-search-tool'

Compare_to_peers = ['LAGGARD','AVERAGE','LEADER']
# Companies = pd.read_csv('russell-3000.csv',skiprows=9,usecols=['Name','Ticker','Sector'])
Companies = pd.read_csv('ftse-allshares.csv',usecols=['Name','Ticker','Industry'])

try:
    # result_sofar = pd.read_csv('MSCI-ESG-Scores-Russel-3000-sofar.csv')
    result_sofar = pd.read_csv('MSCI-ESG-Scores-FTSE-Allshares.csv')
    merged_df = Companies[['Ticker','Name']].merge(result_sofar, left_on='Ticker', right_on='Ticker', how='left', indicator=True)
    rest_companies = merged_df[merged_df['_merge'] == 'left_only']['Ticker'].tolist()
    print('Use existing result')
except:
    result_sofar = pd.DataFrame({ 'Ticker': [], 'Name': []})
    rest_companies = Companies['Name'].tolist()
    print('Start form 0')
    
last_len = 0
skipped_companies = []
Found_companies = []

# 打开目标网页
for i in range(0,10):
    count = 0
    if len(rest_companies)==0:
        break
    try:
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)
        current_soup = main(rest_companies)
        driver.quit()
        rest_companies,result_sofar = Check_Results(result_sofar)
    except Exception as e:
        driver.quit()
        rest_companies,result_sofar = Check_Results(result_sofar)
        print(e)
    if last_len==len(rest_companies):
        break
    last_len = len(rest_companies)
    print("还剩下",last_len,"个公司")


Setting up ChromeDriver service...
Service set up complete.
Starting ChromeDriver with options...
Use existing result
MTRO
Typing search box
Return search box
No search suggestions for keyword 'MTRO', skipping.
CNE
Typing search box
Return search box
No search suggestions for keyword 'CNE', skipping.
SWEF
Typing search box
Return search box
No search suggestions for keyword 'SWEF', skipping.
DIG
Typing search box
Return search box
CLICK!!!!!!!!!!!!!!!!!!
1
WESTERN DIGITAL CORPORATION : aa Finished
BOWL
Typing search box
Return search box
No search suggestions for keyword 'BOWL', skipping.
CPI
Typing search box
Return search box
CLICK!!!!!!!!!!!!!!!!!!
2
CAPITEC BANK HOLDINGS LIMITED : a Finished
PBEE
Typing search box
Return search box
No search suggestions for keyword 'PBEE', skipping.
DLG
Typing search box
Return search box
No search suggestions for keyword 'DLG', skipping.
JFJ
Typing search box
Return search box
No search suggestions for keyword 'JFJ', skipping.
CLI
Typing search bo

KeyboardInterrupt: 